In [1]:
import pandas as pd 
import pickle as pkl 

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
test_fd001 = pd.read_csv(r'C:\potfolio\RUL_Prediction\data\raw\test_FD001.txt', sep= ' ', header= None)
test_fd001.dropna(axis=1,how='all', inplace=True)
scaled_df = pd.read_csv(r'C:\potfolio\RUL_Prediction\data\processed_data\scaled_data.csv')

In [4]:
scaled_df = scaled_df.drop(columns=['RUL'])
cols = scaled_df.columns

In [5]:
test_fd001.columns = cols 
test_fd001.head()

,unit_number,time_in_cycles,oprational_setting_1,oprational_setting_2,oprational_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,21.61,553.90,2388.04,9050.17,1.3,47.20,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,21.61,554.85,2388.01,9054.42,1.3,47.50,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,21.61,554.11,2388.05,9056.96,1.3,47.50,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,21.61,554.07,2388.03,9045.29,1.3,47.28,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,21.61,554.16,2388.01,9044.55,1.3,47.31,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [6]:
len(test_fd001)

13096

In [7]:
# normalize per engine to remove engine-specific bias
def normalize_per_engine(df, sensors):
    df_norm = df.copy()
    for sensor in sensors:
        df_norm[sensor] = df.groupby("unit_number")[sensor].transform(
            lambda x: (x - x.mean()) / x.std()
        )
    return df_norm


def add_rolling_features(df, sensors, windows=[5, 10, 20]):
    df_feat = df.copy()
    for sensor in sensors:
        for w in windows:
            df_feat[f"{sensor}_mean_{w}"] = df.groupby("unit_number")[sensor].transform(
                lambda x: x.rolling(w, min_periods=1).mean()
            )
            df_feat[f"{sensor}_std_{w}"] = df.groupby("unit_number")[sensor].transform(
                lambda x: x.rolling(w, min_periods=1).std()
            )
    return df_feat

def add_diff_features(df, sensors):
    df_feat = df.copy()
    for sensor in sensors:
        df_feat[f"{sensor}_diff"] = df.groupby("unit_number")[sensor].diff().fillna(0)
    return df_feat

**As observed in the feature selection and model-building file, we identified only 6 sensors that contribute significantly to the prediction. Similarly, we will use only these selected sensors as input to the model.**


In [8]:
selected_sensors =  ['sensor_9',  'sensor_11',  'sensor_7',  'sensor_12',  'sensor_14',  'sensor_4']

In [9]:
def feature_engineering_pipeline(df, sensors=selected_sensors):
    df_proc = normalize_per_engine(df, sensors)
    df_proc = add_rolling_features(df_proc, sensors)
    df_proc = add_diff_features(df_proc, sensors)
    return df_proc

test_feat = feature_engineering_pipeline(test_fd001, selected_sensors)

In [10]:
test_feat.head()

,unit_number,time_in_cycles,oprational_setting_1,oprational_setting_2,oprational_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_9_mean_5,sensor_9_std_5,sensor_9_mean_10,sensor_9_std_10,sensor_9_mean_20,sensor_9_std_20,sensor_11_mean_5,sensor_11_std_5,sensor_11_mean_10,sensor_11_std_10,sensor_11_mean_20,sensor_11_std_20,sensor_7_mean_5,sensor_7_std_5,sensor_7_mean_10,sensor_7_std_10,sensor_7_mean_20,sensor_7_std_20,sensor_12_mean_5,sensor_12_std_5,sensor_12_mean_10,sensor_12_std_10,sensor_12_mean_20,sensor_12_std_20,sensor_14_mean_5,sensor_14_std_5,sensor_14_mean_10,sensor_14_std_10,sensor_14_mean_20,sensor_14_std_20,sensor_4_mean_5,sensor_4_std_5,sensor_4_mean_10,sensor_4_std_10,sensor_4_mean_20,sensor_4_std_20,sensor_9_diff,sensor_11_diff,sensor_7_diff,sensor_12_diff,sensor_14_diff,sensor_4_diff
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,-0.852023,14.62,21.61,-0.576462,2388.04,0.256372,1.3,-1.058969,-1.007896,2388.03,-1.685561,8.4052,0.03,392,2388,100.0,38.86,23.3735,0.256372,NaN,0.256372,NaN,0.256372,NaN,-1.058969,NaN,-1.058969,NaN,-1.058969,NaN,-0.576462,NaN,-0.576462,NaN,-0.576462,NaN,-1.007896,NaN,-1.007896,NaN,-1.007896,NaN,-1.685561,NaN,-1.685561,NaN,-1.685561,NaN,-0.852023,NaN,-0.852023,NaN,-0.852023,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,-1.568866,14.62,21.61,1.660272,2388.01,1.233151,1.3,1.863409,0.743403,2388.06,2.662624,8.3803,0.03,393,2388,100.0,39.02,23.3916,0.744761,0.690687,0.744761,0.690687,0.744761,0.690687,0.402220,2.066433,0.402220,2.066433,0.402220,2.066433,0.541905,1.581610,0.541905,1.581610,0.541905,1.581610,-0.132246,1.238355,-0.132246,1.238355,-0.132246,1.238355,0.488531,3.074631,0.488531,3.074631,0.488531,3.074631,-1.210445,0.506885,-1.210445,0.506885,-1.210445,0.506885,0.976779,2.922378,2.236734,1.751299,4.348184,-0.716843
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,-0.047823,14.62,21.61,-0.082026,2388.05,1.816920,1.3,1.863409,-0.012839,2388.03,-0.279431,8.4441,0.03,393,2388,100.0,39.08,23.4166,1.102148,0.788479,1.102148,0.788479,1.102148,0.788479,0.889283,1.687236,0.889283,1.687236,0.889283,1.687236,0.333928,1.174950,0.333928,1.174950,0.333928,1.174950,-0.092444,0.878359,-0.092444,0.878359,-0.092444,0.878359,0.232544,2.218843,0.232544,2.218843,0.232544,2.218843,-0.822904,0.760940,-0.822904,0.760940,-0.822904,0.760940,0.583769,0.000000,-1.742298,-0.756243,-2.942055,1.521043
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1.257397,14.62,21.61,-0.176205,2388.03,-0.865200,1.3,-0.279668,-2.361172,2388.05,0.585879,8.3917,0.03,391,2388,100.0,39.00,23.3737,0.610311,1.175619,0.610311,1.175619,0.610311,1.175619,0.597045,1.496481,0.597045,1.496481,0.597045,1.496481,0.206395,0.992672,0.206395,0.992672,0.206395,0.992672,-0.659626,1.342060,-0.659626,1.342060,-0.659626,1.342060,0.320878,1.820271,0.320878,1.820271,0.320878,1.820271,-0.302829,1.211583,-0.302829,1.211583,-0.302829,1.211583,-2.682120,-2.143077,-0.094178,-2.348333,0.865310,1.305220
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,0.101198,14.62,21.61,0.035697,2388.01,-1.035274,1.3,0.012569,0.703601,2388.03,-0.452493,8.4031,0.03,390,2388,100.0,38.99,23.4130,0.281194,1.256244,0.281194,1.256244,0.281194,1.256244,0.480150,1.322087,0.480150,1.322087,0.480150,1.322087,0.172255,0.863062,0.172255,0.863062,0.172255,0.863062,-0.386981,1.312449,-0.386981,1.312449,-0.386981,1.312449,0.166203,1.613896,0.166203,1.613896,0.166203,1.613896,-0.222023,1.064705,-0.222023,1.064705,-0.222023,1.064705,-0.170074,0.292238,0.211901,3.064773,-1.038372,-1.156199


In [11]:
test_feat.shape

(13096, 68)

In [12]:
from sklearn.impute import SimpleImputer 

imputer = SimpleImputer(strategy= 'mean')
test_feat_imputed = imputer.fit_transform(test_feat)

In [13]:
test_feat_imputed.shape

(13096, 68)

In [14]:
with open(r'C:\potfolio\RUL_Prediction\models\stack_model.pkl', 'rb') as file:
    stack_model = pkl.load(file)

stack_model

,estimators,"[('xgb', ...), ('rf', ...), ...]"
,final_estimator,"XGBRegressor(...obs=None, ...)"
,cv,3
,n_jobs,-1
,passthrough,False
,verbose,0
,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None


In [15]:
y_pred = stack_model.predict(test_feat_imputed)
y_pred

array([218.56706  , 137.42572  , 131.28052  , ...,   7.9229026,
        10.193046 ,   5.838184 ], dtype=float32)

In [16]:
len(y_pred)

13096

In [17]:
# Get last cycle per engine
last_cycles = test_feat.groupby('unit_number').last().reset_index()

# Select the same features used during training
X_test_final = last_cycles

# Impute missing values if needed
X_test_final_imputed = imputer.transform(X_test_final)

# Predict
y_pred_final = stack_model.predict(X_test_final_imputed)

print(len(y_pred_final)) 


100


In [18]:
import numpy as np
# Load true RUL values
true_rul = np.loadtxt(r'C:\potfolio\RUL_Prediction\data\raw\RUL_FD001.txt')

# Compare
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(true_rul, y_pred_final )
rmse = np.sqrt(mse)
print("Test RMSE:", rmse)


Test RMSE: 56.36661000720311


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
import torch
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.relu = nn.LeakyReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# define input dimension
input_dim = 68

# recreate model
model = SimpleNN(input_dim=input_dim)

# load state dictionary (the correct way)
state_dict = torch.load(r"C:\potfolio\RUL_Prediction\notebooks\nn_rul_model_state_2.pth")

model.load_state_dict(state_dict)   # this step was missing
model.eval()                        # switch to eval mode


C:\Users\Asus\AppData\Local\Temp\ipykernel_14416\1814316670.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(r"C:\potfolio\RUL_Prediction\noteboo

SimpleNN(
  (fc1): Linear(in_features=68, out_features=512, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=1, bias=True)
)

In [21]:
# Get last cycle per engine
last_cycles = test_feat.groupby('unit_number').last().reset_index()

# Select the same features used during training
test_feat_imputed = last_cycles

# Impute missing values if needed
test_feat_imputed = imputer.transform(test_feat_imputed)

len(test_feat_imputed)

100

In [22]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# test_feat_imputed is already a NumPy array
X_test_tensor = torch.tensor(test_feat_imputed, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

y_pred = y_pred_tensor.cpu().numpy().flatten()

print("Predictions shape:", y_pred.shape)
print("Sample predictions:", y_pred[:10])


Predictions shape: (100,)
Sample predictions: [54.990982 23.698536 12.737735 25.387453 22.75321  24.379112 21.11713
 23.33551  39.26236  16.001606]


In [23]:
len(y_pred)

100

In [24]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(true_rul, y_pred))
print("RMSE:", rmse)


RMSE: 63.82887329349237


In [ ]:
import plotly.graph_objects as go 

fig = go.Figure()

# Predicted RUL
fig.add_trace(
    go.Scatter(
        x=list(range(len(y_pred_final))),  # Sample index
        y=y_pred_final,
        mode='markers',
        marker=dict(color='blue', size=8),
        name='Predicted RUL'
    )
)

# True RUL
fig.add_trace(
    go.Scatter(
        x=list(range(len(true_rul))),
        y=true_rul,
        mode='markers',
        marker=dict(color='red', size=8),
        name='True RUL'
    )
)

fig.update_layout(
    title='Predicted vs True RUL',
    xaxis_title='Sample Index',
    yaxis_title='RUL',
    template='plotly_white'
)

fig.show()

In [1]:
import os 
report_directory = r'C:\potfolio\RUL_Prediction\reports'
rulfile = 'Predicted vs True RUL(testing) - FD001.svg'
rulpath = os.path.join(report_directory,rulfile)
# fig.write_image(rulpath)

In [25]:
import plotly.graph_objects as go 

fig = go.Figure()

# Predicted RUL
fig.add_trace(
    go.Scatter(
        x=list(range(len(y_pred))),  # Sample index
        y=y_pred,
        mode='markers',
        marker=dict(color='blue', size=8),
        name='Predicted RUL'
    )
)

# True RUL
fig.add_trace(
    go.Scatter(
        x=list(range(len(true_rul))),
        y=true_rul,
        mode='markers',
        marker=dict(color='red', size=8),
        name='True RUL'
    )
)

fig.update_layout(
    title='Predicted vs True RUL',
    xaxis_title='Sample Index',
    yaxis_title='RUL',
    template='plotly_white'
)

fig.show()